# Быстрый style transfer

![](https://miro.medium.com/max/2166/1*8bbp3loQjkLXaIm_QBfD8w.jpeg)

В этом ноутбуке мы реализуем алгоритм для переноса стиля.

[Классический](https://arxiv.org/abs/1508.06576) алгоритм для style transfer работает медленно: для того чтобы его применить к одному изображению, нужно прогнать целую процедуру оптимизации. Идея для его ускорения тривиальна, как всё в deep learning: предсказывать результат этой процедуры оптимизации отдельной нейросетью.

![](https://user-images.githubusercontent.com/37034031/42068027-830719f4-7b84-11e8-9e87-088f1e476aab.png)

Эта идея была предложена в работе 2016 года под названием ["Perceptual Losses for Real-Time Style Transfer and Super-Resolution"](https://arxiv.org/abs/1603.08155). Код для ноутбука основан на примере [`fast_neural_style`](https://github.com/pytorch/examples/tree/master/fast_neural_style) из репозитория `pytorch/examples`.



In [ ]:
import io
import requests
import os
import datetime
from collections import namedtuple
from pathlib import Path
from typing import Optional

from tqdm.notebook import tqdm
from PIL import Image

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets
from torchvision import models
from torchvision import transforms

In [ ]:
%load_ext tensorboard

In [ ]:
np.random.seed(42)
torch.manual_seed(42)

In [ ]:
def torch_image_to_numpy(image_torch):
    """Convert PyTorch tensor to Numpy array.
    :param image_torch: [0..1]-normalized PyTorch float CHW Tensor.
    :returns: Numpy uint8 HWC array in range [0..255].
    """
    assert len(image_torch.shape) == 3, 'Have you forgotten to remove the batch dimension?'
    image_np = image_torch.permute(1, 2, 0).numpy()
    image_np = image_np * 255 + 0.5
    image_np = np.clip(image_np, 0, 255)
    image_np = image_np.astype(np.uint8)
    return image_np

def get_datetime():
    return datetime.datetime.now().isoformat(sep='_', timespec='milliseconds').replace(':', '-')

## Датасет

Для обучения генератора картинок нам понадобится датасет. Подойдёт более-менее любой датасет, в котором распределение картинок похоже на то, на котором мы потом будем применять сетку. Здесь я предлагаю использовать датасет [Flickr8k](http://hockenmaier.cs.illinois.edu/8k-pictures.html), который выложен на [Kaggle](https://www.kaggle.com/adityajn105/flickr8k).

In [ ]:
data_root = Path('flickr8k')

if not data_root.exists():
    !gdown https://drive.google.com/uc?id=1DEEYahajtFjxkWXdRHp5hXF1H952tAD2
    !unzip -q flickr8k.zip -d $data_root
    assert data_root.exists()

Датасет вообще-то предназначается для image captioning, то есть он содержит пары из картинок и текста, описывающего эту картинку. Текст мы использовать не будем.

Давайте посмотрим на этот датасет глазами.

In [ ]:
!ls -l $data_root

In [ ]:
!ls {data_root}/Images | wc -l

А вот так выглядят аннотации, которые мы не будем использовать:

In [ ]:
!head {data_root}/captions.txt

В Torchvision есть удобный класс `ImageFolder`, который позволяет загружать разного рода картиночные датасеты. В нашем случае можно его использовать примерно так:

In [ ]:
dataset = datasets.ImageFolder(data_root)
dataset[0][0]

Но нам понадобится, как обычно, сделать некоторую предобработку этих картинок, прежде чем мы сможем передать их на вход в нейросеть. Давайте в этот раз сделаем такую предобработку:

* Масштабирование так, чтобы меньшая сторона изображения стала длиной 256 пикселей, с сохранением соотношения сторон;
* Вырезание квадрата из центра изображения;
* Преобразование `PIL.Image` в `torch.Tensor`.

In [ ]:
image_size = 256

transform = transforms.Compose([
    <YOUR CODE>
    # Почему мы не делаем .to(device) прямо в transform:
    # https://discuss.pytorch.org/t/to-device-gives-an-error-when-used-inside-transforms-compose/51387
])

dataset = datasets.ImageFolder(data_root, transform=transform)
dataset

Как обычно, заведём dataloader:

In [ ]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=True)

## Картинка со стилем

In [ ]:
def get_image(url):
    response = requests.get(url)
    return Image.open(io.BytesIO(response.content))

In [ ]:
style_url = 'https://github.com/pytorch/examples/raw/master/fast_neural_style/images/style-images/mosaic.jpg'
style_image = get_image(style_url)
style_image

In [ ]:
style_tensor = transform(style_image)
Image.fromarray(torch_image_to_numpy(style_tensor))

## Картинка для валидации

Во время обучения сетки-генератора мы будем периодически проверять текущее качество на этой картинке:

In [ ]:
content_url = 'https://github.com/pytorch/examples/raw/master/fast_neural_style/images/content-images/amber.jpg'

In [ ]:
content_image = get_image(content_url)
content_image

In [ ]:
content_tensor = transform(content_image)
Image.fromarray(torch_image_to_numpy(content_tensor))

## VGG

In [ ]:
!nvidia-smi

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

Для обучения понадобится прогонять то, что сгенерировал генератор, через сеть VGG-16, и извлекать из неё промежуточные представления. Вспомним, что такое VGG-16:

In [ ]:
# Сразу укажем pretrained=True, чтобы веса скачались заранее
vgg16 = models.vgg16(pretrained=True)
vgg16

На семинаре мы будем использовать промежуточные представления, которые получаются перед **первыми 4 макс-пулингами**, сразу после ReLU.

Это не означает, что нельзя использовать другие. Вы можете поэкспериментировать с любыми представлениями!

In [ ]:
# Какие у этих ReLU индексы?
layer_indices = [3, 8, 15, 22]

In [ ]:
VggOutputs = namedtuple("VggOutputs", ['relu1_2', 'relu2_2', 'relu3_3', 'relu4_3'])


class Vgg16(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        # Для удобства загоним прямо внутрь нашего класса imagenet-нормализацию,
        # чтобы потом о ней не думать
        self.register_buffer('imagenet_mean', torch.tensor([0.485, 0.456, 0.406]).reshape(1, 3, 1, 1))
        self.register_buffer('imagenet_std', torch.tensor([0.229, 0.224, 0.225]).reshape(1, 3, 1, 1))
        
        # Приготовьтесь к тому, чтобы в forward() выдавать значения промежуточных слоёв
        <YOUR CODE>

        for p in self.parameters():
            p.requires_grad = False

    def forward(self, X):
        h = (X - self.imagenet_mean) / self.imagenet_std

        # Вычислите активации на 4 выбранных слоях и дайте им названия, как у аргументов VggOutputs
        <YOUR CODE>
        
        out = VggOutputs(h_relu1_2, h_relu2_2, h_relu3_3, h_relu4_3)
        return out


vgg = Vgg16()
vgg = vgg.to(device)
features = vgg(torch.randn(1, 3, 224, 224).to(device))
features.relu4_3

## Генератор

Перейдём к самой интересной части — image-to-image сетке, генерирующей стилизованные изображения. В качестве базового кирпичика в ней мы будем использовать последовательность из свёртки, нормализации и активации. Также опционально этот кирпичик будет делать увеличивать картинку перед свёрткой, что понадобится нам во второй половине генератора.

В качестве нормализации здесь мы будем использовать не batch normalization, а так называемый instance normalization. Сравните:

Batch normalization:

```python
    x - x.mean(dim=(0, 2, 3))
y = -------------------------
      x.var(dim=(0, 2, 3))
```

Instance normalization:

```python
    x - x.mean(dim=(2, 3))
y = ----------------------
      x.var(dim=(2, 3))
```

(Разумеется, это не точная формулировка обеих нормализаций: здесь опущены нюансы про biased/unbiased variance estimation, никак не упоминается скользящее среднее, нет ничего про аффинное преобразование после нормализации, отсутствует ε в знаменателе и так далее. Цель здесь — это показать разницу между двумя нормализациями. Технические детали можете посмотреть тут: [batch norm](https://github.com/dniku/dl-norms/blob/master/dl_norms/batch_norm.py), [instance norm](https://github.com/dniku/dl-norms/blob/master/dl_norms/instance_norm.py).)

In [ ]:
def conv_norm_act(in_channels, out_channels, kernel_size, stride=1, upsample : Optional[int] = None, norm=True, relu=True):
    layers = []
    if upsample is not None:
        # An upsample followed by a convolution gives better results compared to ConvTranspose2d.
        # ref: http://distill.pub/2016/deconv-checkerboard/
        layers.append(nn.Upsample(mode='nearest', scale_factor=upsample))
    layers.append(nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding=kernel_size // 2))
    if norm:
        layers.append(nn.InstanceNorm2d(out_channels, affine=True))
    if relu:
        layers.append(nn.ReLU())
    return nn.Sequential(*layers)

Кирпичик мы будем использовать как по отдельности, так и в составе residual-блока, как в резнете. Давайте опишем такой блок.

```
----> [conv(3x3)->norm->relu] --> [conv(3x3)->norm] --> + -->
  |                                                     ↑
  |_____________________________________________________|
```

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        <YOUR CODE>

    def forward(self, x):
        <YOUR CODE>

Наконец, опишем сам генератор. Дадим ему такую структуру:

```
Вход: картинка с 3 каналами

image ->
[conv(32,9x9)->norm->relu] ->
[conv(64,3x3,stride=2)->norm->relu] ->
    [conv(128,3x3,stride=2)->norm->relu] ->
        (5 раз) ResidualBlock ->
        [upsample(x2)->conv(64,3x3)->norm->relu] ->
    [upsample(x2)->conv(32,3x3)->norm->relu] ->
[upsample(x2)->conv(3,9x9)] ->
sigmoid ->
stylized image
```

Вопрос: как определяются отступы строк в этой схеме?

In [ ]:
transformer = <YOUR CODE>
transformer = transformer.to(device)
transformer

Посмотрим, что необученный генератор выдаёт на первой картинке из датасета.

In [ ]:
Image.fromarray(np.hstack([
    torch_image_to_numpy(dataset[0][0]),
    torch_image_to_numpy(transformer(dataset[0][0].unsqueeze(0).to(device)).squeeze(0).detach().cpu()),
]))

## Обучение

### Content loss

Лосс в style transfer состоит из двух частей. Первая из них — content loss, отвечающий за то, чтобы сохранять семантику картинки. Также его называют feature reconstruction loss или perceptual loss. Формула для него такая:

$$
l_{\text{content}}(\hat y, y) = \frac 1 {B C_j H_j W_j} || \text{VGG}_j(\hat y) - \text{VGG}_j (y) ||_2^2
$$

Здесь $\hat y$ — батч сгенерированных изображений, $y$ — батч из content images, VGG подразумевает уже подготовленную нами VGG-16, а $j$ — это индекс слоя в ней. Мы будем использовать $j = \text{relu2_2}$. $B$, $C_j$, $H_j$ и $W_j$ — это размеры соответствующего тензора активаций.

In [ ]:
def get_content_loss(gen_features, content_features):
    <YOUR CODE>

### Style loss

Вторая компонента лосса — это style loss (или style reconstruction loss). Он записывается немного сложнее:

$$
l_{\text{style}}(\hat y, y) = \sum_{j} \frac 1 {B C_j^2} \left|\left| \frac 1 {C_j H_j W_j} \text{Gram}_j(\hat y) - \frac 1 {C_j H_j W_j} \text{Gram}_j (y) \right|\right|_2^2
$$

Здесь $\text{Gram}_j$ — это матрица Грама для активаций $j$-го слоя. Вообще, матрица Грама — это матрица, состоящая из скалярных произведений. Чтобы объяснить, какие скалярные произведения имеются в виду в этом случае, вспомним, что для каждого элемента батча активации $j$-го слоя имеют форму $C_j \times H_j \times W_j$. Теперь представим, что $f$ — это результат решейпа тензора активаций в форму $C_j \times H_j \cdot W_j$. Тогда матрица Грама — это такая матрица:

```
                        |----------|
                        |   C_j    |
    |---------------|   |          |   |----------|
    |   W_j * H_j   |   |          |   |   C_j    |
    |C_j            | x |W_j * H_j | = |C_j       |
    |               |   |          |   |          |
    |---------------|   |          |   |----------|
                        |          |
                        |----------|
```

То есть для каждого канала мы считаем скалярное произведение с каждым другим каналом по пространственным размерностям. Идея здесь в том, что это полностью уничтожает всю пространственную информацию.

Мы будем использовать style loss на всех слоях, которые мы подготовили в нашем шаблоне `Vgg16`.

Примечание: вполне возможно, что от такого количества коэффициентов можно избавиться, потюнив вес style loss в общем лоссе. Я позаимствовал эту схему из примера [`fast_neural_style`](https://github.com/pytorch/examples/tree/master/fast_neural_style).

In [ ]:
def gram_matrix(t):
    b, c, h, w = t.shape

    # Реализуйте вычисление матрицы Грама.
    # Это можно сделать многими способами. Возможно, вам понадобятся какие-то из этих функций:
    # transpose, bmm, matmul, einsum

    <YOUR CODE>
    
    return gram / (c * h * w)

In [ ]:
def get_style_loss(gen_gram_matrices, style_gram_matrices):
    # {gen,style}_gram_matrices — это списки из 4 элементов, каждый из которых является матрицей Грама
    <YOUR CODE>

### Обучающий цикл

In [ ]:
opt = torch.optim.Adam(transformer.parameters(), lr=1e-3)

In [ ]:
def train(transformer, opt, style_tensor, content_tensor, dataloader, vgg, device, tb_dir,
          epochs=2, content_weight=1e5, style_weight=1e10):
    style_tensor = style_tensor.repeat(dataloader.batch_size, 1, 1, 1)  # to avoid a warning in F.mse_loss
    style_tensor = style_tensor.to(device)

    # Посчитайте список из матриц Грама для стилевой картинки
    style_gram_matrices = <YOUR CODE>

    content_tensor = content_tensor.unsqueeze(0).to(device)
    
    batch_idx = 0
    
    with SummaryWriter(log_dir=str(tb_dir / get_datetime())) as writer:
        for e in range(epochs):
            for content_batch, _ in tqdm(dataloader):
                content_batch = content_batch.to(device)

                # Пропустите батч через генератор
                gen_batch = <YOUR CODE>

                # Посчитайте активации VGG на сгенерированных картинках
                gen_features = <YOUR CODE>
                content_features = <YOUR CODE>

                # Посчитайте content loss
                content_loss = <YOUR CODE>

                # Посчитайте матрицы Грама по активациям сгенерированных картинок
                gen_gram_matrices = [gram_matrix(f) for f in gen_features]

                # Это нужно, чтобы избежать проблем на последнем батче
                style_gram_matrices_truncated = [
                    style_gram_matrix[:content_batch.shape[0]] for style_gram_matrix in style_gram_matrices
                ]

                # Посчитайте style loss (используя style_gram_matrices_truncated)
                style_loss = <YOUR CODE>

                # Посчитайте итоговый лосс с весами content_weight и style_weight
                total_loss = <YOUR CODE>

                # Сделайте шаг оптимизации
                <YOUR CODE>
                
                writer.add_scalar('losses/content', content_loss.item(), batch_idx)
                writer.add_scalar('losses/style', style_loss.item(), batch_idx)
                writer.add_scalar('losses/total', total_loss.item(), batch_idx)

                if (batch_idx + 1) % 100 == 0:
                    transformer.eval()
                    with torch.no_grad():
                        y = transformer(content_tensor)
                        writer.add_image('image', y.detach().squeeze(0), batch_idx)
                    transformer.train()
                
                batch_idx += 1

In [ ]:
tb_dir = Path('tb_logs')

In [ ]:
%tensorboard --port 6007 --logdir $tb_dir

In [ ]:
train(transformer, opt, style_tensor, content_tensor, dataloader, vgg, device, tb_dir)

На всякий случай ещё раз посмотрим, что наша модель выдаёт на валидационной картинке.

In [ ]:
Image.fromarray(np.hstack([
    torch_image_to_numpy(content_tensor),
    torch_image_to_numpy(transformer(content_tensor.unsqueeze(0).to(device)).squeeze(0).detach().cpu()),
]))

А теперь погоняем нашу модель на разных картинках из интернета.

In [ ]:
def stylize(url, factor=4):
    image = get_image(url)

    w, h = image.size
    new_w = w // factor * factor
    new_h = h // factor * factor

    # Почему тут необходим CenterCrop? Почему factor именно 4?
    transform_no_resize = transforms.Compose([
        transforms.CenterCrop((new_h, new_w)),
        transforms.ToTensor()
    ])

    image_tensor = transform_no_resize(image)
    assert image_tensor.shape[1:] == (new_h, new_w)

    with torch.no_grad():
        stylized_tensor = transformer(image_tensor.unsqueeze(0).to(device)).squeeze(0).cpu()

    assert image_tensor.shape == stylized_tensor.shape

    return Image.fromarray(
        np.hstack([
            torch_image_to_numpy(image_tensor),
            torch_image_to_numpy(stylized_tensor),
        ])
    )

In [ ]:
stylize(content_url)

In [ ]:
stylize('https://upload.wikimedia.org/wikipedia/commons/thumb/e/ec/Mona_Lisa%2C_by_Leonardo_da_Vinci%2C_from_C2RMF_retouched.jpg/515px-Mona_Lisa%2C_by_Leonardo_da_Vinci%2C_from_C2RMF_retouched.jpg')

In [ ]:
stylize('https://data.whicdn.com/images/93462738/original.jpg')

Кажется, что на больших картинках модель рисует очень мелкие детали витража — да и вообще эти детали имеют более-менее одинаковый размер в пикселях. А как бы сделать так, чтобы эти детали были разного размера?